In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
# Import libraries
import pandas as pd
import os
from pathlib import Path
from tqdm import tqdm
import yaml
import matplotlib.pyplot as plt
from ultralytics import YOLO
import numpy as np
from PIL import Image, ExifTags
import torch

In [3]:
# INPUT_DIRS
INPUT_DATA_DIR = Path('dataset')
## Drop the Folder if it already exists
DATASETS_DIR = Path('dataset')
# Image & labels directory
TRAIN_IMAGES_DIR = DATASETS_DIR / 'images' / 'train'
TRAIN_LABELS_DIR = DATASETS_DIR / 'labels'/ 'train'
TEST_IMAGES_DIR = DATASETS_DIR / 'images' / 'test'
VAL_IMAGES_DIR = DATASETS_DIR / 'images' /'val'
VAL_LABELS_DIR = DATASETS_DIR / 'labels' /'val'

# Load train and test files
train = pd.read_csv(INPUT_DATA_DIR / 'Train_df.csv')
val = pd.read_csv(INPUT_DATA_DIR / 'Val_df.csv')
test = pd.read_csv(INPUT_DATA_DIR / 'Test.csv')
ss = pd.read_csv(INPUT_DATA_DIR / 'SampleSubmission.csv')

class_map = {cls: i for i, cls in enumerate(sorted(train['class'].unique().tolist()))}
# Strip any spacing from the class item and make sure that it is a str
train['class'] = train['class'].str.strip()

# Map {'healthy': 2, 'cssvd': 1, anthracnose: 0}
train['class_id'] = train['class'].map(class_map)

train_df = train
val_df = val

# Create a data.yaml file required by yolo
class_names = sorted(train['class'].unique().tolist())
num_classes = len(class_names)
data_yaml = {
	"path" : str(DATASETS_DIR.absolute()),
	'train': str(TRAIN_IMAGES_DIR.absolute()),
	'val': str(VAL_IMAGES_DIR.absolute()),
	'test': str(TEST_IMAGES_DIR.absolute()),
	'nc': num_classes,
	'names': class_names
}

val_image_names = [str(Path(name).stem) for name in val_df['Image_ID'].unique()]
train_image_names = [str(Path(name).stem) for name in train['ImagePath'].unique()]

In [4]:
val_df['Image_ID'].nunique()

56

In [5]:
from glob import glob

# Validate the model on the validation set
BEST_PATH = sorted(glob("runs/detect/train*/weights/best.pt"))[-1]
BEST_PATH

'runs/detect/train2/weights/best.pt'

In [6]:
# Load the trained YOLO model
model = YOLO(BEST_PATH)

model = model.eval()

In [7]:
for flag, v in ExifTags.TAGS.items():
    if v == "Orientation":
        break


def load_image(filepath):
    image = Image.open(filepath)
    return image

    exif = image._getexif()
    if exif is None:
        return image

    orientation_value = exif.get(flag, None)

    if orientation_value == 3:
        image = image.rotate(180, expand=True)
    elif orientation_value == 6:
        image = image.rotate(270, expand=True)
    elif orientation_value == 8:
        image = image.rotate(90, expand=True)
    return image


flag

274

In [8]:
v

'Orientation'

In [10]:
# Path to the test images directory
test_dir_path = VAL_IMAGES_DIR

# Get a list of all image files in the test directory
image_files = os.listdir(test_dir_path)

# Initialize an empty list to store the results for all images
all_data = []

# Initialize an empty list to store the results for all images
all_data = []

# Iterate through each image in the directory
for image_file in tqdm(image_files):
	# Full path to the image
	img_path = os.path.join(test_dir_path, image_file)

	# Make predictions on the image
	results = model.predict(
		load_image(img_path),
        conf=0.0,
        imgsz=1024,
        # max_det=100,
        verbose=False,
        # nms=True,
        # iou=0.7,
        # nbs=64,
        device="cuda:0",
	)  # verbose=False,

	# Extract bounding boxes, confidence scores, and class labels
	boxes = (
		results[0].boxes.xyxy.tolist() if results[0].boxes else []
	)  # Bounding boxes in xyxy format
	classes = results[0].boxes.cls.tolist() if results[0].boxes else []  # Class indices
	confidences = (
		results[0].boxes.conf.tolist() if results[0].boxes else []
	)  # Confidence scores
	names = results[0].names  # Class names dictionary

	if boxes:  # If detections are found
		for box, cls, conf in zip(boxes, classes, confidences):
			x1, y1, x2, y2 = box
			detected_class = names[
				int(cls)
			]  # Get the class name from the names dictionary

			# Add the result to the all_data list
			all_data.append(
				{
					"Image_ID": str(image_file),
					"class": detected_class,
					"confidence": conf,
					"ymin": y1,
					"xmin": x1,
					"ymax": y2,
					"xmax": x2,
				}
			)
	else:  # If no objects are detected
		all_data.append(
			{
				"Image_ID": str(image_file),
				"class": "None",
				"confidence": None,
				"ymin": None,
				"xmin": None,
				"ymax": None,
				"xmax": None,
			}
		)

100%|██████████| 56/56 [00:09<00:00,  6.03it/s]


In [11]:
# Convert the list to a DataFrame for all images
sub = pd.DataFrame(all_data)

In [12]:
sub.head()

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
0,ID_MYSxE2.jpg,healthy,0.722172,265.918793,398.916077,3264.000000,1930.119995
1,ID_MYSxE2.jpg,healthy,0.113201,0.000000,1755.506714,724.512268,2446.072021
2,ID_MYSxE2.jpg,healthy,0.095444,0.000000,1480.353882,940.120361,2448.000000
3,ID_MYSxE2.jpg,healthy,0.063643,0.000000,1554.322021,608.756592,2444.166016
4,ID_MYSxE2.jpg,healthy,0.061984,0.000000,1272.819702,559.004578,1857.875122


In [13]:
sub.describe()

,confidence,ymin,xmin,ymax,xmax
count,16800.000000,16800.000000,16800.000000,16800.000000,16800.000000
mean,0.004851,689.698684,611.149425,1238.050780,1067.879025
std,0.041051,883.192715,753.374775,1106.001250,912.720689
min,0.000027,0.000000,0.000000,9.327269,8.373480
25%,0.000182,2.231561,2.880145,367.747780,362.108322
50%,0.000416,353.930649,342.214874,958.283905,814.787598
75%,0.001114,1005.935776,864.178711,1734.282806,1520.022644
max,0.817452,3983.311523,3905.188965,4128.000000,4032.000000


In [14]:
sub['class'].value_counts()

class
cssvd          7194
healthy        4871
anthracnose    4735
Name: count, dtype: int64

In [15]:
def load_yolo_labels(label_folder):
	label_data = {}
	label_folder = Path(label_folder)

	for label_file in label_folder.glob("*"):
		with open(label_file, "r") as file:
			annotations = []
			for line in file:
				parts = line.strip().split()
				if len(parts) == 5:
					class_id, x_center, y_center, width, height = map(float, parts)
					annotations.append({
						"class_id": int(class_id),
						"x_center": x_center,
						"y_center": y_center,
						"width": width,
						"height": height
					})
				else:
					print(f"Skipping line in {label_file}: {line.strip()}")
			label_data[label_file.stem] = annotations
	# Convert the label data to a pandas DataFrame
	label_df = []
	for image_id, annotations in label_data.items():
		for annotation in annotations:
			label_df.append({
				"Image_ID": image_id,
				"class_id": annotation["class_id"],
				"x_center": annotation["x_center"],
				"y_center": annotation["y_center"],
				"width": annotation["width"],
				"height": annotation["height"]
			})

	label_df = pd.DataFrame(label_df)
	return label_df

# Example usage
label_folder = VAL_LABELS_DIR
labels = load_yolo_labels(label_folder)
labels.sample(5)

def yolo_to_bbox(image_folder, labels_df: pd.DataFrame):
	image_folder = Path(image_folder)
	converted_bboxes = []
	for image_file in image_folder.glob("*"):
		image_id = image_file.stem
		if image_id not in labels_df['Image_ID'].values:
			converted_bboxes.append({
				"Image_ID": image_id,
				"class_id": -1,  # Indicating no label
				"xmin": None,
				"ymin": None,
				"xmax": None,
				"ymax": None
			})

	for _, row in labels_df.iterrows():
		all_ids = list(image_folder.glob(f"{row['Image_ID']}.*"))
		image_path = image_folder / f"{row['Image_ID']}"
		if all_ids:
			image_path = all_ids[0]

		if image_path.exists():
			img = load_image(image_path)
			img_width, img_height = img.size

			x_center = row['x_center'] * img_width
			y_center = row['y_center'] * img_height
			width = row['width'] * img_width
			height = row['height'] * img_height

			x_min = x_center - (width / 2)
			y_min = y_center - (height / 2)
			x_max = x_center + (width / 2)
			y_max = y_center + (height / 2)

			converted_bboxes.append({
				"Image_ID": row['Image_ID'],
				"class_id": row['class_id'],
				"xmin": x_min,
				"ymin": y_min,
				"xmax": x_max,
				"ymax": y_max
			})
		else:
			print(f"Image {image_path} not found.")

	return pd.DataFrame(converted_bboxes)

# Example usage
converted_labels = yolo_to_bbox(VAL_IMAGES_DIR, labels)
converted_labels.sample(5)

,Image_ID,class_id,xmin,ymin,xmax,ymax
29,ID_E2z7VZ,1,552.000000,303.000320,960.000000,1036.000000
28,ID_E2z7VZ,1,18.000000,21.000320,769.000320,1174.000000
5,ID_OB1kf9,0,115.000320,51.999360,960.000000,1279.999360
52,ID_t1Jamz,0,512.999424,402.001488,3033.999360,1993.000464
77,ID_ka3SdB,2,0.999856,1.999920,348.000016,397.999888


In [16]:
converted_labels.describe()

,class_id,xmin,ymin,xmax,ymax
count,95.000000,95.000000,95.000000,95.000000,95.000000
mean,1.126316,358.452658,331.631492,1269.147347,1588.589391
std,0.775175,538.727657,481.446497,950.695543,1184.230438
min,0.000000,-0.002016,-0.001632,58.999824,53.000064
25%,1.000000,1.000392,20.500032,498.499968,704.499664
50%,1.000000,114.999840,114.000000,960.000000,1195.000320
75%,2.000000,477.500328,513.499776,1862.000208,2442.499248
max,2.000000,2514.001344,2346.998784,3897.998496,4128.002064


In [17]:
converted_labels["Image_ID"].nunique()

56

In [18]:
converted_labels['class_id'].value_counts()

class_id
1    37
2    35
0    23
Name: count, dtype: int64

In [19]:
class_map

{'anthracnose': 0, 'cssvd': 1, 'healthy': 2}

In [20]:
id_class_map = {v: k for k, v in class_map.items()}
converted_labels['class'] = converted_labels['class_id'].map(id_class_map)
converted_labels['class'].value_counts()

class
cssvd          37
healthy        35
anthracnose    23
Name: count, dtype: int64

In [21]:
converted_labels.sample(5)

,Image_ID,class_id,xmin,ymin,xmax,ymax,class
40,ID_gqbwtB,2,11.999936,1.000064,274.000064,53.000064,healthy
28,ID_E2z7VZ,1,18.000000,21.000320,769.000320,1174.000000,cssvd
65,ID_kOXMTE,2,2415.000960,113.998464,3225.001728,547.999488,healthy
36,ID_RUW7ek,2,86.999744,79.000064,361.999872,416.000000,healthy
91,ID_T4Tw1d,1,75.999840,11.000320,816.999840,1280.000000,cssvd


In [22]:
sub.sample(5)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
9478,ID_QEGas6.jpg,anthracnose,0.000139,2.933655,758.909607,276.211853,959.995728
5625,ID_Hm1rUc.jpeg,cssvd,0.000379,657.118103,1351.167358,1597.979858,2723.610840
9550,ID_QEGas6.jpg,anthracnose,0.000098,0.000000,17.661514,95.115967,820.592163
10481,ID_ka3SdB.jpg,healthy,0.000128,400.630371,385.731781,415.747192,415.284149
5997,ID_TSkjv2.jpg,cssvd,0.000058,150.240387,0.000000,609.515442,390.093262


In [23]:
sub.loc[:, "Image_ID"] = sub["Image_ID"].apply(lambda x: str(Path(x).stem))

sub.sample(3)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
3238,ID_gqbwtB,healthy,0.000698,265.176605,223.935577,319.030701,312.679474
13736,ID_crD9na,healthy,0.000148,2575.909180,2125.058594,3251.809326,2443.834717
2865,ID_v1PbJC,anthracnose,0.001241,0.000000,1356.535522,198.519974,1526.641113


In [24]:
def convert_df(df: pd.DataFrame):
	df = df.copy().dropna()
	return {
		img_id: {
			"boxes": torch.tensor(raw[["xmin", "ymin", "xmax", "ymax"]].values, dtype=torch.float32),
			"scores": (
				torch.tensor(raw["confidence"].values, dtype=torch.float32)
				if "confidence" in raw.columns
				else None
			),
			"labels": torch.tensor(raw["class_id"].values, dtype=torch.int32),
		}
		for (img_id, ), raw in df.groupby(["Image_ID"])
	}

def default_value():
	return {
		"boxes": torch.empty((0, 4), dtype=torch.float32),
		"scores": torch.empty((0,), dtype=torch.float32),
		"labels": torch.empty((0,), dtype=torch.int32),
	}

def get_preds_data(preds, thr: float = 0.5):
	if thr is not None:
		preds = preds[preds["confidence"] >= thr]
	preds = convert_df(preds)
	d = default_value()
	return {i: preds.get(i, d) for i in converted_labels["Image_ID"].unique()}

In [25]:
converted_labels.isna().sum()

Image_ID    0
class_id    0
xmin        0
ymin        0
xmax        0
ymax        0
class       0
dtype: int64

In [26]:
ground_truth = convert_df(converted_labels)
ground_truth = {k: ground_truth[k] for k in converted_labels["Image_ID"].unique()}

len(ground_truth)

56

In [28]:
import torch

def calculate_iou_tensor(box1, box2):
	"""
	box1: [4], box2: [4]
	Format: [xmin, ymin, xmax, ymax]
	"""
	xA = torch.max(box1[0], box2[0])
	yA = torch.max(box1[1], box2[1])
	xB = torch.min(box1[2], box2[2])
	yB = torch.min(box1[3], box2[3])

	inter_area = torch.clamp(xB - xA, min=0) * torch.clamp(yB - yA, min=0)
	box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
	box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
	union_area = box1_area + box2_area - inter_area
	return inter_area / union_area if union_area > 0 else torch.tensor(0.0)

def evaluate_detection(predictions, ground_truths, iou_threshold=0.5, conf_threshold=0.0):
	"""
	predictions: list of dicts (len = batch size), each dict with 'boxes', 'scores', 'labels'
	ground_truths: list of dicts with 'boxes', 'labels'
	"""
	TP = 0
	FP = 0
	FN = 0

	for preds, gts in zip(predictions, ground_truths):
		pred_boxes = preds['boxes']
		pred_labels = preds['labels']
		pred_scores = preds['scores'] if preds['scores'] is not None else torch.ones(len(pred_boxes))

		gt_boxes = gts['boxes']
		gt_labels = gts['labels']
		matched_gt = set()

		for i in range(len(pred_boxes)):
			if pred_scores[i] < conf_threshold:
				continue
			pred_box = pred_boxes[i]
			pred_label = pred_labels[i]
			match_found = False

			for j in range(len(gt_boxes)):
				if j in matched_gt:
					continue
				if pred_label != gt_labels[j]:
					continue
				iou = calculate_iou_tensor(pred_box, gt_boxes[j])
				if iou >= iou_threshold:
					TP += 1
					matched_gt.add(j)
					match_found = True
					break
			if not match_found:
				FP += 1

		FN += len(gt_boxes) - len(matched_gt)

	precision = TP / (TP + FP) if (TP + FP) else 0.0
	recall = TP / (TP + FN) if (TP + FN) else 0.0
	f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) else 0.0
	accuracy = TP / (TP + FP + FN) if (TP + FP + FN) else 0.0

	return {
		'TP': TP,
		'FP': FP,
		'FN': FN,
		'Precision': precision,
		'Recall': recall,
		'F1 Score': f1_score,
		'Accuracy': accuracy
	}

In [29]:
sub.sample(3)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
2890,ID_v1PbJC,healthy,0.001073,2393.298096,1982.089600,3257.775879,2443.557373
6756,ID_K6JkTA,cssvd,0.000618,3169.935303,1267.133057,3264.000000,1542.170654
12879,ID_dA1GQh,cssvd,0.000043,230.875320,0.111681,471.212219,23.068550


In [30]:
sub["class_id"] = sub["class"].map(class_map)

sub.sample(3)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
5418,ID_Hm1rUc,cssvd,0.005947,443.393494,2483.786621,1545.323853,3023.517334,1
3081,ID_gqbwtB,healthy,0.006460,333.243561,122.574898,408.082916,223.851624,2
10446,ID_ka3SdB,healthy,0.000156,0.000000,0.000000,37.000294,23.991112,2


In [31]:
predictions = get_preds_data(sub, None)

len(predictions)

56

In [32]:
for i in np.linspace(0.0, 0.95, 15):
	scores = evaluate_detection(
		predictions.values(),
		ground_truth.values(),
		iou_threshold=0.5,
		conf_threshold=i
	)
	print("Evaluation metric at:", i, " score :", scores)

Evaluation metric at: 0.0  score : {'TP': 93, 'FP': 16707, 'FN': 2, 'Precision': 0.005535714285714285, 'Recall': 0.9789473684210527, 'F1 Score': 0.011009174311926604, 'Accuracy': 0.005535055350553505}
Evaluation metric at: 0.06785714285714285  score : {'TP': 80, 'FP': 77, 'FN': 15, 'Precision': 0.5095541401273885, 'Recall': 0.8421052631578947, 'F1 Score': 0.634920634920635, 'Accuracy': 0.46511627906976744}
Evaluation metric at: 0.1357142857142857  score : {'TP': 76, 'FP': 28, 'FN': 19, 'Precision': 0.7307692307692307, 'Recall': 0.8, 'F1 Score': 0.763819095477387, 'Accuracy': 0.6178861788617886}
Evaluation metric at: 0.20357142857142857  score : {'TP': 69, 'FP': 16, 'FN': 26, 'Precision': 0.8117647058823529, 'Recall': 0.7263157894736842, 'F1 Score': 0.7666666666666667, 'Accuracy': 0.6216216216216216}
Evaluation metric at: 0.2714285714285714  score : {'TP': 63, 'FP': 12, 'FN': 32, 'Precision': 0.84, 'Recall': 0.6631578947368421, 'F1 Score': 0.7411764705882353, 'Accuracy': 0.5887850467289

In [33]:
converted_labels.sample(5)

,Image_ID,class_id,xmin,ymin,xmax,ymax,class
60,ID_bJrfR5,1,343.999656,47.998944,2901.000312,4031.997984,cssvd
2,ID_jTWHzU,0,-0.001512,24.998400,2483.999784,4032.000000,anthracnose
71,ID_wFa3gD,2,525.000528,666.001248,1874.000304,3074.999712,healthy
64,ID_kOXMTE,2,2514.001344,566.000856,2885.000640,1120.000392,healthy
5,ID_OB1kf9,0,115.000320,51.999360,960.000000,1279.999360,anthracnose


In [34]:
import torch
from torchmetrics.detection import MeanAveragePrecision

def compute_map(preds, targets, iou_thresholds):
	"""
	Compute mAP at different IoU thresholds using torchmetrics.
	
	Args:
		preds: List of dicts with 'boxes', 'scores', 'labels' for predictions
		targets: List of dicts with 'boxes', 'labels' for ground truth
		iou_thresholds: List of IoU thresholds to evaluate
	
	Returns:
		Dict containing mAP results for each IoU threshold
	"""
	# Initialize the metric
	metric = MeanAveragePrecision(iou_thresholds=iou_thresholds)
	
	# Update metric with predictions and targets
	metric.update(preds, targets)
	
	# Compute the results
	result = metric.compute()
	
	return result

thrs = np.linspace(0.0, 0.95, 15)
# Example usage
# if __name__ == "__main__":
# Example predictions and targets
iou_thresholds = [0.5]
for i in thrs:
	preds = list(get_preds_data(sub, i).values())

	targets = list(ground_truth.values())

	# Compute mAP
	results = compute_map(preds, targets, iou_thresholds)

	# Print results
	print("mAP Results:", i, " - ", results)

/data/home/eak/learning/nganga_ai/AminiCocoa/Amini-Cocoa-Contamination-Challenge/.venv/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: Encountered more than 100 detections in a single image. This means that certain detections with the lowest scores will be ignored, that may have an undesirable impact on performance. Please consider adjusting the `max_detection_threshold` to suit your use case. To disable this warning, set attribute class `warn_on_many_detections=False`, after initializing the metric.
  warnings.warn(*args, **kwargs)


mAP Results: 0.0  -  {'map': tensor(0.8051), 'map_50': tensor(0.8051), 'map_75': tensor(-1.), 'map_small': tensor(-1.), 'map_medium': tensor(-1.), 'map_large': tensor(0.8115), 'mar_1': tensor(0.5444), 'mar_10': tensor(0.9103), 'mar_100': tensor(0.9719), 'mar_small': tensor(-1.), 'mar_medium': tensor(-1.), 'mar_large': tensor(0.9719), 'map_per_class': tensor(-1.), 'mar_100_per_class': tensor(-1.), 'classes': tensor([0, 1, 2], dtype=torch.int32)}
mAP Results: 0.06785714285714285  -  {'map': tensor(0.7797), 'map_50': tensor(0.7797), 'map_75': tensor(-1.), 'map_small': tensor(-1.), 'map_medium': tensor(-1.), 'map_large': tensor(0.7836), 'mar_1': tensor(0.5444), 'mar_10': tensor(0.8558), 'mar_100': tensor(0.8558), 'mar_small': tensor(-1.), 'mar_medium': tensor(-1.), 'mar_large': tensor(0.8558), 'map_per_class': tensor(-1.), 'mar_100_per_class': tensor(-1.), 'classes': tensor([0, 1, 2], dtype=torch.int32)}
mAP Results: 0.1357142857142857  -  {'map': tensor(0.7571), 'map_50': tensor(0.7571), 

In [35]:
sub.to_csv('dataset/evaluations/validation.csv', index=False)

In [36]:
import pandas as pd

sub = pd.read_csv('dataset/evaluations/validation.csv')
sub.sample(5)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
2048,ID_t1Jamz,anthracnose,0.000136,2718.354248,1140.174438,3017.308594,2054.877441,0
10762,ID_RUW7ek,healthy,0.000366,26.567717,82.168686,79.678848,154.313965,2
16115,ID_aTTp2e,cssvd,0.000220,0.671208,1696.657715,130.746429,2026.297852,1
4038,ID_jTqcfA,anthracnose,0.000660,1150.968506,194.879532,1277.109863,353.323914,0
4640,ID_ydI3Pa,cssvd,0.001284,489.998932,1544.739990,997.062195,2088.553223,1
